In [1]:
import tensorflow as tf
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
 
# setting path
sys.path.append('../')

In [4]:
from bbox_utils import *
from classification_loss import *

In [5]:
h = tf.keras.losses.Huber(delta=float('inf'), reduction=tf.keras.losses.Reduction.NONE)

In [6]:
actual_deltas =  tf.constant(
    [
        [
            [-0.1, -0.05, 0, 0.091, 0.],
            [0.1, 0.0022, 0.25, 0.14, 1.]
        ],
        [
            [0.1, -0.05, 0, 0.091, 0.],
            [0.1, 0.0022, 0.25, 0.14, 1.]
        ],
        [
            [0.1, -0.05, 0, 0.091, 1.],
            [0.1, 0.0022, 0.25, 0.14, 1.]
        ]
    ]
)

pred_deltas = tf.constant(
    [
        [
            [-0.05, 0.05, 0.1, 0.2],
            [0.1, 0.2, -0.1, 0.02]
        ],
        [
            [-0.05, 0.05, 0.1, 0.2],
            [0.1, 0.2, -0.1, 0.02]
        ],
        [
            [-0.05, 0.05, 0.1, 0.2],
            [0.1, 0.2, -0.1, 0.02]
        ]
    ]
)

loss = h(actual_deltas[:, :, :4], pred_deltas)


2022-08-26 15:17:43.559649: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
actual_deltas.shape

TensorShape([3, 2, 5])

In [8]:
loss

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.00429763, 0.0220031 ],
       [0.00679763, 0.0220031 ],
       [0.00679763, 0.0220031 ]], dtype=float32)>

In [9]:
loss = loss * actual_deltas[:, :, 4]

In [10]:
loss

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.        , 0.0220031 ],
       [0.        , 0.0220031 ],
       [0.00679763, 0.0220031 ]], dtype=float32)>

In [11]:
tf.reduce_sum(loss, axis=-1)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.0220031 , 0.0220031 , 0.02880073], dtype=float32)>

In [12]:
huber_loss(actual_deltas[:, :, :4], pred_deltas)

NameError: name 'huber_loss' is not defined

In [ ]:
h(y_true, y_pred)

In [ ]:
pos = tf.constant([[0.,1.]])

In [ ]:
tf.expand_dims(pos, -1)

In [ ]:
x = tf.constant([
    [
        [-0.05, 0.05, 0.1, 0.2],
        [0.1, 0.2, -0.1, 0.02]
    ]
])

In [ ]:
x

In [ ]:
y = tf.concat([ x, tf.expand_dims(pos, -1) ], axis = 2)

In [ ]:
y

In [ ]:
y[:, :, :4]

In [ ]:
i, j = 3, 0

box_start_i = (i // 3) * 3
box_end_i = (i // 3) * 3 + 3

box_start_j = (j // 3) * 3
box_end_j = (j // 3) * 3 + 3

box_start_i, box_end_i, box_start_j, box_end_j

In [13]:
feature_map_shapes = [38, 19, 10, 5, 3, 1]
aspect_ratios = [
    [1, 2, 0.5],
    [1, 2, 3, 0.5, 0.333],
    [1, 2, 3, 0.5, 0.333],
    [1, 2, 3, 0.5, 0.333],
    [1, 2, 0.5],
    [1, 2, 0.5]
]
number_of_feature_maps = 6

In [14]:
prior_boxes = generate_default_boxes(feature_map_shapes, number_of_feature_maps, aspect_ratios)
boxes = convert_to_box_form(prior_boxes)

In [15]:
gt_boxes = tf.constant([[
    [0., 0., 0.2, 0.2],
    [0.8, 0.8, 1.0, 0.9]
]
])

gt_labels = tf.constant([
            [1, 2]
])

threshold = 0.5

In [16]:
gt_boxes_map_offset_suppressed_with_pos_cond, gt_labels_one_hot_encoded = match_priors_with_gt(prior_boxes, boxes, gt_boxes, gt_labels, number_of_labels = 21, threshold = 0.5)

In [17]:

tf.reduce_sum(gt_boxes_map_offset_suppressed_with_pos_cond[:, :, 4], axis=1)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([52.], dtype=float32)>

In [18]:
gt_labels_one_hot_encoded = tf.concat( [ gt_labels_one_hot_encoded, gt_labels_one_hot_encoded ], axis = 0)
gt_boxes_map_offset_suppressed_with_pos_cond = tf.concat( [ gt_boxes_map_offset_suppressed_with_pos_cond, gt_boxes_map_offset_suppressed_with_pos_cond ], axis = 0)

In [19]:
gt_boxes_map_offset_suppressed_with_pos_cond + tf.random.normal(shape = gt_boxes_map_offset_suppressed_with_pos_cond.shape)

<tf.Tensor: shape=(2, 8732, 5), dtype=float32, numpy=
array([[[-0.27508184,  0.02872645, -0.72557646, -0.25859258,
          1.2054785 ],
        [ 0.55546963,  1.1746279 , -0.3035527 ,  2.0280447 ,
          0.28704867],
        [ 0.42370242, -1.7518297 ,  0.87277067,  2.008859  ,
         -1.3742366 ],
        ...,
        [-1.3645357 ,  0.09975508, -0.46381623,  0.47390595,
          0.00559981],
        [-0.35607934,  1.0143555 , -0.3305083 , -0.29026183,
         -0.5582955 ],
        [-0.35275117,  0.27585188, -1.8501881 , -0.97125804,
          0.61734915]],

       [[ 2.2412522 ,  1.2843703 , -0.09314153,  0.38468102,
          1.2180057 ],
        [-0.67206347, -1.9196736 ,  0.18871701, -0.09246585,
          1.7443793 ],
        [-1.5857825 , -1.5857729 ,  2.7819483 ,  1.0615819 ,
          0.36899117],
        ...,
        [ 1.2275034 ,  0.8135524 , -0.288911  ,  0.2728582 ,
          1.5594393 ],
        [ 0.5987493 , -1.2183837 ,  0.01698157, -0.29446265,
          0.45761

In [20]:
pred_labels = tf.random.uniform(shape = (1, 8732, 21))

In [27]:
gt_boxes_map_offset_suppressed_with_pos_cond.shape

TensorShape([2, 8732, 5])

In [22]:
pred_labels = pred_labels / tf.math.reduce_sum(
    pred_labels, axis = 1
)

In [23]:
pred_labels = tf.concat( [ pred_labels, pred_labels ], axis = 0)

In [29]:
pred_delta = gt_boxes_map_offset_suppressed_with_pos_cond[:, :, :4] + tf.random.normal(shape = (2, 8732, 4) )

In [33]:
total_pos_boxes = tf.reduce_sum(gt_boxes_map_offset_suppressed_with_pos_cond[:, :, 4:], axis=1)

total_pos_boxes

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[52.],
       [52.]], dtype=float32)>

In [40]:
loss = SSDLoss()

In [41]:
loc_loss = loss.localization_loss(gt_boxes_map_offset_suppressed_with_pos_cond, 
                       gt_boxes_map_offset_suppressed_with_pos_cond[:, :, :4] + 
                       tf.random.normal(shape = (2, 8732, 4) ))

In [42]:
loss.confidence_loss(gt_labels_one_hot_encoded, pred_labels)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([25.539293, 25.539293], dtype=float32)>

In [ ]:
categorical_cross_entropy = tf.losses.CategoricalCrossentropy(
            reduction=tf.losses.Reduction.NONE
        )

In [ ]:
confidence_loss_for_all = categorical_cross_entropy(gt_labels_one_hot_encoded, pred_labels)

In [ ]:
# gt_labels_one_hot_encoded
pos_cond = tf.reduce_any( tf.equal(gt_labels_one_hot_encoded[..., 1:], tf.constant(1.0)), axis = 2)
pos_mask = tf.cast(pos_cond, dtype=tf.float32)
total_pos_boxes = tf.reduce_sum(pos_mask, axis=1)

In [ ]:
pos_loss = tf.reduce_sum(pos_mask * confidence_loss_for_all, axis=1)

In [ ]:
pos_cond

In [ ]:
tf.where(neg_cond, confidence_loss_for_all, tf.negative(tf.ones(confidence_loss_for_all.shape)))

In [ ]:
neg_cond = tf.reduce_any( tf.equal(gt_labels_one_hot_encoded[..., :1], tf.constant(1.0)), axis = 2)
neg_mask = tf.cast(neg_cond, dtype=tf.float32)

In [ ]:
# set positive cases to -1
confidence_loss_for_all = tf.where(neg_cond, confidence_loss_for_all, 
                                   tf.negative(tf.ones(confidence_loss_for_all.shape))
                                  )

In [ ]:
tf.reduce_sum(neg_mask, axis=1)

In [ ]:
total_neg_bboxes = tf.cast(total_pos_boxes * 3, tf.int32)

In [ ]:
total_neg_bboxes

In [ ]:
loss_sorted_indices = tf.argsort(confidence_loss_for_all, direction="DESCENDING")
loss_sorted_rank = tf.argsort(loss_sorted_indices)

In [ ]:
loss_sorted_rank

In [ ]:
neg_cond = tf.less(loss_sorted_rank, tf.expand_dims(total_neg_bboxes, axis=1))
neg_mask = tf.cast(neg_cond, dtype=tf.float32)

In [ ]:
neg_mask

In [ ]:
conf_loss = tf.reduce_sum(neg_mask * confidence_loss_for_all, axis=1) + pos_loss

In [ ]:
conf_loss

In [ ]:
pos_loss

In [ ]:
tf.reduce_sum(neg_mask * confidence_loss_for_all, axis=1)

In [ ]:
total_pos_boxes